In [59]:
import cv2
import numpy as np

class ImageAligner:
    def __init__(self, image_data):
        self.image_data = image_data

    def align_image_by_contour(self):
        # Загрузка изображения
        image = cv2.imread(self.image_data)
        if image is None:
            raise ValueError("Image not found or not readable")

        # Преобразование в оттенки серого
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Пример: выравнивание по контуру через детектор границ Canny
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Выбираем самый большой контур
        contour = max(contours, key=cv2.contourArea)

        # Вычисляем ориентацию контура
        rect = cv2.minAreaRect(contour)
        angle = rect[2]

        # Корректировка угла
        if angle < -45:
            angle = -(90 + angle)
        elif angle < 0:
            angle = -angle

        # Применяем поворот
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        aligned_image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        return aligned_image

    def align_image_by_content(self):
        # Загрузка изображения
        image = cv2.imread(self.image_data)
        if image is None:
            raise ValueError("Image not found or not readable")

        # Пример: выравнивание по содержимому через ключевые точки SIFT
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(image, None)

        # Извлечение координат ключевых точек
        keypoints_coords = np.array([keypoint.pt for keypoint in keypoints])

        # Выбор опорных точек
        # Предположим, что у нас есть опорные точки для изображения в выходном размере
        reference_points = np.array([[0, 0], [0, image.shape[1]], [image.shape[0], 0], [image.shape[0], image.shape[1]]])

        # Вычисление гомографии с помощью RANSAC
        ransacReprojThreshold = 4
        H, mask = cv2.findHomography(keypoints_coords, reference_points, cv2.RANSAC, ransacReprojThreshold)

        # Применение преобразования к изображению
        aligned_image = cv2.warpPerspective(image, H, (image.shape[1], image.shape[0]))

        return aligned_image

image_file = "Project2.jpg"
image_aligner = ImageAligner(image_file)

# Выравнивание изображения по контуру
aligned_image_contour = image_aligner.align_image_by_contour()
cv2.imshow("Aligned Image by Contour", aligned_image_contour)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Выравнивание изображения по содержимому
aligned_image_content = image_aligner.align_image_by_content()
cv2.imshow("Original Image", aligned_image_content)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\fundam.cpp:389: error: (-215:Assertion failed) src.checkVector(2) == dst.checkVector(2) in function 'cv::findHomography'
